In [ ]:
# Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("elyza/Llama-3-ELYZA-JP-8B")
# model = AutoModelForCausalLM.from_pretrained("elyza/Llama-3-ELYZA-JP-8B")

# Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B")
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-3B")


# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)

# Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("Qwen/QwQ-32B-Preview")
# model = AutoModelForCausalLM.from_pretrained("Qwen/QwQ-32B-Preview")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 17/17 [00:08<00:00,  1.96it/s]


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig

model_name = "elyza/Llama-3-ELYZA-JP-8B"
# model_name = "Qwen/Qwen2.5-3B"
# model_name = "microsoft/Phi-3.5-mini-instruct"
# model_name = "Qwen/QwQ-32B-Preview"
tokenizer = AutoTokenizer.from_pretrained(model_name)

quant_config = BitsAndBytesConfig(
    # load_in_8bit=True  # 8ビット量子化を指定,
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    
    bnb_4bit_compute_dtype=torch.bfloat16 #bfloatはfloatに比べて、精度よりも数値が表現できる桁数の幅（ダイナミックレンジ）を重視した設計...らしい
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config, #量子化を行う
    torch_dtype="auto",
    device_map="auto",
)
model.eval()

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:12<00:00,  3.02s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [3]:


DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。特に指示が無い場合は、常に日本語で回答してください。"
text = "仕事の熱意を取り戻すためのアイデアを5つ挙げてください。"
assistant_output = "そんなもんねーよハゲ！"


messages = [
    {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
    {"role": "user", "content": text},
    {"role": "assistant", "content": assistant_output},
]
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
token_ids = tokenizer.encode(
    prompt, add_special_tokens=False, return_tensors="pt"
)

print(prompt)
print(token_ids)
# with torch.no_grad():
#     output_ids = model.generate(
#         token_ids.to(model.device),
#         max_new_tokens=1200,
#         do_sample=True,
#         temperature=0.6,
#         top_p=0.9,
#     )
# output = tokenizer.decode(
#     output_ids.tolist()[0][token_ids.size(1):], skip_special_tokens=True
# )
# print(output)


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

あなたは誠実で優秀な日本人のアシスタントです。特に指示が無い場合は、常に日本語で回答してください。<|eot_id|><|start_header_id|>user<|end_header_id|>

仕事の熱意を取り戻すためのアイデアを5つ挙げてください。<|eot_id|><|start_header_id|>assistant<|end_header_id|>

そんなもんねーよハゲ！<|eot_id|><|start_header_id|>assistant<|end_header_id|>


tensor([[128000, 128006,   9125, 128007,    271,  30591, 112568,  15682, 124097,
         103350,  16556, 104622, 106241,  26854, 102433, 107707,  39880,  57207,
         105335,  52414,  38641,   1811,  66378,  20230,  64467,  20379,  29295,
          43568,  16995, 126513,   5486,  40053,  20230, 102433, 102158,  16556,
         113925,  39926,  72315,   1811, 128009, 128006,    882, 128007,    271,
         117876,  16144, 107969,  37689,  30512, 108167, 114941,  17663, 124122,
         111090,  68408,  39880,  30512,     20,  59739, 123680, 102639,  38144,
          72315,   1811, 128009, 128006,  78191, 128007,    271, 107943,  32977,
          25827, 101832,  38248,    